# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Model, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import joblib
from azureml.data.dataset_factory import TabularDatasetFactory

## Dataset

In [2]:
breast_cancer_url = 'https://raw.githubusercontent.com/JPSvdW/Machine_Learning_Engineer_with_Microsoft_Azure_Nanodegree_Capstone_Project/main/data/breast-cancer.csv'
breast_cancer_dataset = TabularDatasetFactory.from_delimited_files(breast_cancer_url)

breast_cancer_dataframe = breast_cancer_dataset.to_pandas_dataframe()

breast_cancer_dataframe.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
breast_cancer_dataframe.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [4]:
ws = Workspace.from_config()
experiment_name = 'breast_cancer_hyperdrive_experiment'

experiment=Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
breast_cancer_hyperdrive_experiment,quick-starts-ws-173266,Link to Azure Machine Learning studio,Link to Documentation


## Create compute cluster

In [5]:
my_compute_cluster_name = "bc-cluster"

try:
    my_compute_target = ComputeTarget(workspace = ws, name = my_compute_cluster_name)
    print('An existing cluster has been found, please use it.')

except ComputeTargetException:
    my_compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes = 4, min_nodes = 1)
    my_compute_target = ComputeTarget.create(ws, my_compute_cluster_name, my_compute_config)

my_compute_target.wait_for_completion(show_output=True)
print(my_compute_target.get_status().serialize())

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded......................
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2022-01-07T08:28:33.390000+00:00', 'errors': None, 'creationTime': '2022-01-07T08:26:48.895645+00:00', 'modifiedTime': '2022-01-07T08:26:54.573171+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [6]:
early_termination_policy = BanditPolicy(evaluation_interval = 1, slack_factor = 0.1, delay_evaluation = 4)

my_param_sampling = RandomParameterSampling({"--C": uniform(0.1, 1),"--max_iter": choice(50, 100, 150, 200, 250)})

my_estimator = SKLearn(source_directory = "./", compute_target = my_compute_target, vm_size="STANDARD_D2_V2", entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(
    estimator=my_estimator,
    hyperparameter_sampling=my_param_sampling,
    policy=early_termination_policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=25,
    max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


## Submit Experiment

In [7]:
my_hyperdrive_run = experiment.submit(hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(my_hyperdrive_run).show()
my_hyperdrive_run.get_status()
my_hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_43cecc15-3c48-4a93-a6d9-ac158c17591a
Web View: https://ml.azure.com/runs/HD_43cecc15-3c48-4a93-a6d9-ac158c17591a?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-173266/workspaces/quick-starts-ws-173266&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-01-07T08:29:59.396517][API][INFO]Experiment created<END>\n""<START>[2022-01-07T08:30:00.461331][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-01-07T08:30:01.170806][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_43cecc15-3c48-4a93-a6d9-ac158c17591a
Web View: https://ml.azure.com/runs/HD_43cecc15-3c48-4a93-a6d9-ac158c17591a?wsid=/subscriptions/1b944a9b-fdae-4f97-aeb1-b7eea0beac53/resourcegroups/aml-quickstarts-173266/workspaces/quick-starts-ws-173266&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



{'runId': 'HD_43cecc15-3c48-4a93-a6d9-ac158c17591a',
 'target': 'bc-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-01-07T08:29:59.110014Z',
 'endTimeUtc': '2022-01-07T08:44:30.634992Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "Accuracy", "goal": "maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'a1ef0a63-4211-45e6-b1ff-16e1d01a22ef',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'space_size': 'infinite_space_size',
  'score': '0.9649122807017544',
  'best_child_run_id': 'HD_43cecc15-3c48-4a93-a6d9-ac158c17591a_1',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://mlstrg173266.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_43cecc15-3c48-4a93-a6d9-ac158c

KeyError: 'log_files'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [9]:
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
my_hyperdrive_best_run = my_hyperdrive_run.get_best_run_by_primary_metric()

print("Details of my best run: ", my_hyperdrive_best_run.get_details(),"/n")
print("File names of my best run: ", my_hyperdrive_best_run.get_file_names(), "/n")
print("Metrics of my best run: ", my_hyperdrive_best_run.get_metrics())

Details of my best run:  {'runId': 'HD_43cecc15-3c48-4a93-a6d9-ac158c17591a_1', 'target': 'bc-cluster', 'status': 'Completed', 'startTimeUtc': '2022-01-07T08:31:00.804591Z', 'endTimeUtc': '2022-01-07T08:32:04.186133Z', 'services': {}, 'warnings': [{'message': 'This run might be using a new job runtime with improved performance and error reporting. The logs from your script are in user_logs/std_log.txt. Please let us know if you run into any issues, and if you would like to opt-out, please add the environment variable AZUREML_COMPUTE_USE_COMMON_RUNTIME to the environment variables section of the job and set its value to the string "false"'}], 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': 'a1ef0a63-4211-45e6-b1ff-16e1d01a22ef', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}, 'inputDatasets': [], 'outputDatasets': [], 'runDefinition': {'script': 'train.py', 'command': '', 'useAbsolutePath': Fa

In [10]:
print('Hyperdrive best run: ', my_hyperdrive_best_run)
print('\n Best run accuracy: ', my_hyperdrive_best_run.get_metrics()['Accuracy'])
print('\n')
print(my_hyperdrive_best_run.get_details()['runDefinition']['arguments'])

Hyperdrive best run:  Run(Experiment: breast_cancer_hyperdrive_experiment,
Id: HD_43cecc15-3c48-4a93-a6d9-ac158c17591a_1,
Type: azureml.scriptrun,
Status: Completed)

 Best run accuracy:  0.9649122807017544


['--C', '0.31218517931449213', '--max_iter', '50']


## Save best model

In [11]:
best_model_parameter_values = my_hyperdrive_best_run.get_details()['runDefinition']['arguments']

os.makedirs('outputs', exist_ok = True)

joblib.dump(best_model_parameter_values, filename = './outputs/my-best-hyperdrive-model.pkl')

['./outputs/my-best-hyperdrive-model.pkl']

## Register best model

In [13]:
best_model = Model.register(workspace = ws, model_name = 'my-best-hyperdrive-model', model_path = 'outputs/my-best-hyperdrive-model.pkl', 
tags = {'Method' : 'HyperDrive'}, properties = {'Accuracy': my_hyperdrive_best_run.get_metrics()['Accuracy']})

Registering model my-best-hyperdrive-model


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

